# 컴피티션 링크
- https://www.kaggle.com/t/2e45abe9f1434b59a3358365432a48bb

# 고객별 백화점 구매기록 데이터셋
- 미혼(1) or 기혼(0) 고객을 예측
- 학습용 구매기록 데이터
    - https://drive.google.com/file/d/1tg41qXu02FK55bFa8P1Gx2URhwPUCoQQ/view?usp=sharing
- 학습용 정답 데이터
    - https://drive.google.com/file/d/1-9AcU9nAoO4SzSmqdCOYTvWNpA5Pdt3q/view?usp=sharing
- 테스트용 구매기록 데이터
    - https://drive.google.com/file/d/1-AwDfGlHm9rNtpnHIWOK96jBJYy3f2SZ/view?usp=sharing
- 제출 양식 데이터
    - https://drive.google.com/file/d/1-Qv7SlsY5Eu3bRR7Z0IUJyLkOO1Fl3y6/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


- 데이터 경로 변수

In [ ]:
DATA_PATH = '/content/drive/MyDrive/부트캠프/Data/'

DATA_PATH

'/content/drive/MyDrive/부트캠프/Data/'

- 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

# 출력 길이 설정 (너무 길 경우 큰 숫자로 설정 가능)
import pandas as pd
pd.options.display.max_colwidth = None  # 한 행의 최대 글자수 제한 해제
pd.options.display.max_rows = None      # 최대 행 수 제한 해제

# 대분류별 중분류 목록을 확인
category_mapping = train_tr.groupby('대분류')['중분류'].unique()
category_mapping


In [ ]:
from IPython.display import display, HTML
br_html = HTML("<br><br>")
display(HTML("<h1>학습용 구매기록 데이터</h1>"), train_tr.head(), br_html)
display(HTML("<h1>학습용 정답 데이터</h1>"), train_target.head(), br_html)
display(HTML("<h1>테스트용 구매기록 데이터</h1>"), test_tr.head(), br_html)
display(HTML("<h1>제출 양식 데이터</h1>"), submit.head(), br_html)

,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
0,train_13219,2004-05-01 09:40:00,A144000,공산품파트,차류,5100,59700
1,train_5590,2004-05-01 09:40:00,A144000,잡화파트,화장잡화,5101,17000
2,train_7200,2004-05-01 10:20:00,A112000,공산품,용기보증,5100,34937
3,train_3010,2004-05-01 10:30:00,A373000,아동_스포츠,아동복,5105,19000
4,train_10851,2004-05-01 10:30:00,A112000,가정용품,전화기_카세트,5110,215000


,ID,target
0,train_0,1.0
1,train_1,1.0
2,train_2,0.0
3,train_3,0.0
4,train_4,0.0


,ID,구매일시,지점코드,대분류,중분류,브랜드코드,구매가격
0,test_3366,2004-05-01 10:20:00,A373000,생식품,건식품,5100,7000
1,test_9389,2004-05-01 10:23:00,A373000,명품잡화,스타킹,5104,46000
2,test_8190,2004-05-01 10:30:00,A144000,잡화파트,수입종합화장품,5106,260000
3,test_8835,2004-05-01 10:30:00,A144000,잡화파트,준보석,5107,17000
4,test_1167,2004-05-01 10:30:00,A373000,남성의류,셔츠,5108,138000


,ID,target
0,test_0,0.5
1,test_1,0.5
2,test_2,0.5
3,test_3,0.5
4,test_4,0.5


## 날짜 형식으로 변환

In [ ]:
train_tr["구매일시"] = pd.to_datetime(train_tr["구매일시"])
test_tr["구매일시"] = pd.to_datetime(test_tr["구매일시"])

## 새로 만든 feature와 병합할 고객ID로만 이루어진 데이터프레임 생성

In [ ]:
train_ft = train_target[["ID"]]
test_ft = submit[["ID"]]

train_ft.shape, test_ft.shape

((14940, 1), (12225, 1))

## 구매일시를 이용한 특성생성

In [ ]:
agg_list = [
    ('웨딩성수기_구매비율', lambda x: np.mean(x.dt.month.isin([4, 5, 9, 10]))),  # 결혼 성수기 시즌 구매 비율
    ('1월_구매비율', lambda x: np.mean(x.dt.month == 1)), # 1월 구매비율
    ('2월_구매비율', lambda x: np.mean(x.dt.month == 2)), # 2월 구매비율
    ('3월_구매비율', lambda x: np.mean(x.dt.month == 3)), # 3월 구매비율
    ('4월_구매비율', lambda x: np.mean(x.dt.month == 4)), # 4월 구매비율
    ('5월_구매비율', lambda x: np.mean(x.dt.month == 5)), # 5월 구매비율
    ('6월_구매비율', lambda x: np.mean(x.dt.month == 6)), # 6월 구매비율
    ('7월_구매비율', lambda x: np.mean(x.dt.month == 7)), # 7월 구매비율
    ('8월_구매비율', lambda x: np.mean(x.dt.month == 8)), # 8월 구매비율
    ('9월_구매비율', lambda x: np.mean(x.dt.month == 9)), # 9월 구매비율
    ('10월_구매비율', lambda x: np.mean(x.dt.month == 10)), # 10월 구매비율
    ('11월_구매비율', lambda x: np.mean(x.dt.month == 11)), # 11월 구매비율
    ('12월_구매비율', lambda x: np.mean(x.dt.month == 12)), # 12월 구매비율
    ('웨딩성수기_구매횟수', lambda x: ((x.dt.month == 4) | (x.dt.month == 5) | (x.dt.month == 9) | (x.dt.month == 10)).sum()),  # 결혼 성수기 시즌 구매 횟수
    ('1월_방문횟수', lambda x: (x.dt.month == 1).sum()),  # 1월 방문 횟수
    ('2월_방문횟수', lambda x: (x.dt.month == 2).sum()),  # 2월 방문 횟수
    ('3월_방문횟수', lambda x: (x.dt.month == 3).sum()),  # 3월 방문 횟수
    ('4월_방문횟수', lambda x: (x.dt.month == 4).sum()),  # 4월 방문 횟수
    ('5월_방문횟수', lambda x: (x.dt.month == 5).sum()),  # 5월 방문 횟수
    ('6월_방문횟수', lambda x: (x.dt.month == 6).sum()),  # 6월 방문 횟수
    ('7월_방문횟수', lambda x: (x.dt.month == 7).sum()),  # 7월 방문 횟수
    ('8월_방문횟수', lambda x: (x.dt.month == 8).sum()),  # 8월 방문 횟수
    ('9월_방문횟수', lambda x: (x.dt.month == 9).sum()),  # 9월 방문 횟수
    ('10월_방문횟수', lambda x: (x.dt.month == 10).sum()),  # 10월 방문 횟수
    ('11월_방문횟수', lambda x: (x.dt.month == 11).sum()),  # 11월 방문 횟수
    ('12월_방문횟수', lambda x: (x.dt.month == 12).sum())   # 12월 방문 횟수

]

tmp = train_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
train_ft = train_ft.merge(tmp, how='left', on="ID")
train_ft.head()

,ID,내점일수_x,구매주기_x,주말방문비율_x,봄_구매비율_x,여름_구매비율_x,가을_구매비율_x,겨울_구매비율_x,주구매요일_x,일별평균구매건수_x,...,3월_방문횟수_y,4월_방문횟수_y,5월_방문횟수_y,6월_방문횟수_y,7월_방문횟수_y,8월_방문횟수_y,9월_방문횟수_y,10월_방문횟수_y,11월_방문횟수_y,12월_방문횟수_y
0,train_0,16,23,0.250000,0.050000,0.250000,0.400000,0.300000,3,1.818182,...,0,0,1,3,2,0,4,0,4,5
1,train_1,41,16,0.023810,0.357143,0.166667,0.357143,0.119048,3,2.000000,...,6,2,7,1,1,5,10,2,3,3
2,train_2,102,6,0.210526,0.464912,0.140351,0.175439,0.219298,0,2.035714,...,13,21,19,6,7,3,9,5,6,3
3,train_3,191,3,0.189573,0.379147,0.180095,0.236967,0.203791,3,2.293478,...,29,34,17,16,19,3,16,14,20,15
4,train_4,55,11,0.258065,0.112903,0.612903,0.209677,0.064516,4,2.296296,...,2,0,5,10,20,8,9,3,1,0


In [ ]:
tmp = test_tr.groupby('ID')["구매일시"].agg(agg_list).reset_index()
test_ft = test_ft.merge(tmp, how='left',on="ID")
test_ft.head()

,ID,내점일수_x,구매주기_x,주말방문비율_x,봄_구매비율_x,여름_구매비율_x,가을_구매비율_x,겨울_구매비율_x,주구매요일_x,일별평균구매건수_x,...,3월_방문횟수_y,4월_방문횟수_y,5월_방문횟수_y,6월_방문횟수_y,7월_방문횟수_y,8월_방문횟수_y,9월_방문횟수_y,10월_방문횟수_y,11월_방문횟수_y,12월_방문횟수_y
0,test_0,7,32,0.571429,0.285714,0.285714,0.428571,0.000000,1,1.400000,...,0,0,2,1,1,0,0,3,0,0
1,test_1,4,54,0.250000,0.000000,0.000000,0.750000,0.250000,0,1.333333,...,0,0,0,0,0,0,1,2,0,0
2,test_2,4,8,0.000000,0.000000,0.600000,0.400000,0.000000,0,2.500000,...,0,0,0,0,0,3,2,0,0,0
3,test_3,28,16,0.031250,0.062500,0.468750,0.343750,0.125000,3,2.000000,...,0,2,0,0,9,6,3,8,0,3
4,test_4,19,29,0.476190,0.380952,0.285714,0.238095,0.095238,6,2.333333,...,0,0,8,0,6,0,0,5,0,0
